In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import datetime

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from scipy.optimize import curve_fit

from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.cluster import KMeans

from sklearn.cluster import DBSCAN

from sklearn.decomposition import KernelPCA

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from mlxtend.plotting import plot_decision_regions

from scipy import optimize

In [ ]:
# Import data set of 'norne_production_rate_sample.csv'
file_path = 'norne_production_rate_sample.csv'
df = pd.read_csv(file_path)

# Convert date string to Panda datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Display the modified DataFrame
print(df)

In [ ]:
# Select data from June 2, 2004, to May 1, 2006
start_date = '2004-06-02'
end_date = '2006-05-01'
selected_data = df.loc[(df['Date'] >= start_date) & (df['Date'] <= end_date), ['Date', 'Flow Rate']]

# Convert the date data to numpy arrays
date_array = selected_data['Date'].to_numpy()
flow_rate_array = selected_data['Flow Rate'].to_numpy()

# Calculate cumulative sum over timedeltas
cumulative_sum = np.cumsum(np.diff(date_array).astype('timedelta64[s]'))

In [ ]:
# Normalize the time and rate data
selected_data['Normalized Date'] = (selected_data['Date'] - selected_data['Date'].min()) / (selected_data['Date'].max() - selected_data['Date'].min())
selected_data['Normalized Flow Rate'] = (selected_data['Flow Rate'] - selected_data['Flow Rate'].min()) / (selected_data['Flow Rate'].max() - selected_data['Flow Rate'].min())

In [ ]:
# Define the hyperbolic decline function
def hyperbolic(t, qi, di, b):
    return qi / (np.abs((1 + b * di * t))**(1/b))

# Perform curve fitting with the hyperbolic function
params, covariance = curve_fit(hyperbolic, normalized_time, normalized_rate, p0=[1, 0.1, 0.5])

# Extract the fitted parameters
qi_fit, di_fit, b_fit = params

# Generate fitted curve using the fitted parameters
fitted_curve = hyperbolic(normalized_time, qi_fit, di_fit, b_fit)

# Plot the original data and the fitted curve
plt.figure(figsize=(10, 6))
plt.scatter(normalized_time, normalized_rate, label='Original Data')
plt.plot(normalized_time, fitted_curve, label='Fitted Curve', color='red')
plt.xlabel('Normalized Time')
plt.ylabel('Normalized Flow Rate')
plt.legend()
plt.show()

In [ ]:
# De-normalize qi and di
qi = qi_fit * (flow_rate_array.max() - flow_rate_array.min()) + flow_rate_array.min()
di = di_fit / (date_array.max() - date_array.min())

# Forecast gas rate until 1,200 days
forecasted_time = np.linspace(normalized_time.min(), 1200, 100)  # 100 points for better resolution
forecasted_rate = hyperbolic(forecasted_time, qi_fit, di_fit, b_fit)

# Plot the production data with the forecasts (rate and cum. production)
plt.figure(figsize=(12, 8))

# Plotting production rate
plt.subplot(2, 1, 1)
plt.scatter(normalized_time, normalized_rate, label='Original Data')
plt.plot(forecasted_time, forecasted_rate, label='Forecasted Rate', color='red')
plt.xlabel('Normalized Time')
plt.ylabel('Normalized Flow Rate')
plt.legend()

# Calculate cumulative production
cumulative_production = np.cumsum(forecasted_rate) * (date_array.max() - date_array.min()) / 100  # Scaling back to original range

# Plotting cumulative production
plt.subplot(2, 1, 2)
plt.plot(forecasted_time, cumulative_production, label='Cumulative Production', color='green')
plt.xlabel('Normalized Time')
plt.ylabel('Cumulative Production')
plt.legend()

plt.tight_layout()
plt.show()

**Q.2 (7 points) Supervised learning** 

   **Q.2-1 (3 points) Data preparation and preprocessing including scaling** 

In [ ]:
# Import the training data set of 'facies_training_data.csv'
file_path = 'facies_training_data.csv'
df = pd.read_csv(file_path)

# Assuming 'Date' is the name of the column containing date strings
df['Date'] = pd.to_datetime(df['Date'])

# Now 'Date' column is converted to pandas datetime format

# Assuming there's a 'Date' column in the dataset
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Delete the null values, specifically for 'PE'
df = df.dropna(subset=['PE'])

In [ ]:
# Take 'Facies' values as the output values
y = df['Facies']

# Drop unnecessary columns from the feature vectors
# Drop 'Formation', 'Well Name', 'Depth', 'NM_M', 'RELPOS', 'Facies'
# Keep 'GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE'
X = df.drop(['Formation', 'Well Name', 'Depth', 'NM_M', 'RELPOS', 'Facies'], axis=1)

In [ ]:
# Scale the feature vectors using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split the training set and test set (80%-20%, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

   **Q.2-2 (1 point) Logistic Regression** 

In [ ]:
# Perform Logistic Regression with specified hyperparameters
lr_model = LogisticRegression(C=0.1, random_state=1)
lr_model.fit(X_train, y_train)

In [ ]:
# Perform prediction with the test data set
y_pred_LR = lr_model.predict(X_test)

In [ ]:
# Print accuracy by using the following code. 
# y_test is the output of the test set
# y_pred_LR is the prediction of  the test data set, ie, the previous solution.

from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test, y_pred_LR)

In [ ]:
def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i,i]
    acc = total_correct/sum(sum(conf))
    return acc

   **Q.2-3 (1 points) Support Vector Machine** 

In [ ]:

# Support Vector Machine with linear kernel
svm_linear_model = SVC(kernel='linear', C=0.1, random_state=1)
svm_linear_model.fit(X_train, y_train)

# Perform prediction with the test data set (linear kernel)
y_pred_svm_linear = svm_linear_model.predict(X_test)

# Print the confusion matrix (linear kernel)
conf_matrix_linear = confusion_matrix(y_test, y_pred_svm_linear)
print("Confusion Matrix (Linear Kernel):\n", conf_matrix_linear)

# Calculate accuracy using the provided function (linear kernel)
acc_linear = accuracy(conf_matrix_linear)
print("Accuracy (Linear Kernel):", acc_linear)

In [ ]:
# Perform prediction with the test data set (Gaussian kernel)
y_pred_svm_rbf = svm_rbf_model.predict(X_test)

# Print the confusion matrix (Gaussian kernel)
conf_matrix_rbf = confusion_matrix(y_test, y_pred_svm_rbf)
print("\nConfusion Matrix (Gaussian Kernel):\n", conf_matrix_rbf)

# Calculate accuracy using the provided function (Gaussian kernel)
acc_rbf = accuracy(conf_matrix_rbf)
print("Accuracy (Gaussian Kernel):", acc_rbf)

**Q. 2-4 (1 points) Decision Tree & Random forest** 

In [ ]:
# Decision Tree Classifier
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=6, random_state=1)
dt_model.fit(X_train, y_train)

# Perform prediction with the test data set
y_pred_dt = dt_model.predict(X_test)

# Print the confusion matrix
conf_matrix_dt = confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix (Decision Tree):\n", conf_matrix_dt)

# Calculate accuracy using the provided function
acc_dt = accuracy(conf_matrix_dt)
print("Accuracy (Decision Tree):", acc_dt)

In [ ]:
# Random Forest Classifier
rf_model = RandomForestClassifier(criterion='gini', max_depth=4, n_estimators=20, random_state=1, n_jobs=2)
rf_model.fit(X_train, y_train)

# Perform prediction with the test data set
y_pred_rf = rf_model.predict(X_test)

# Print the confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix (Random Forest):\n", conf_matrix_rf)

# Calculate accuracy using the provided function
acc_rf = accuracy(conf_matrix_rf)
print("Accuracy (Random Forest):", acc_rf)

**Q. 2-5 (1 point) KNN** 

In [ ]:
# K-Neighbors Classifier
knn_model = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn_model.fit(X_train, y_train)

# Perform prediction with the test data set
y_pred_knn = knn_model.predict(X_test)

# Print the confusion matrix
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)
print("Confusion Matrix (K-Neighbors Classifier):\n", conf_matrix_knn)

# Calculate accuracy using the provided function
acc_knn = accuracy(conf_matrix_knn)
print("Accuracy (K-Neighbors Classifier):", acc_knn)

**Q. 3 (3 points) Clustering** 

**Q. 3-1 (1.5 point) KMeans Clustering** 

In [ ]:
# Take the iris data set for clustering. 
from sklearn import datasets
iris=datasets.load_iris()
X_train=iris.data[:,[2,3]]

In [ ]:
# Feature Scaling
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

In [ ]:
# Perform KMeans clustering
km = KMeans(n_clusters=3, init='random', n_init=100, tol=1e-04, random_state=0)
y_km = km.fit_predict(X_train_std)

In [ ]:
# Run the followign code for visualization.
# y_km is the results vector after clustering.

plt.scatter(X_train_std[y_km==0,0],X_train_std[y_km==0,1],c='lightgreen',marker='s',edgecolor='black',s=50,label='Cluster 1')
plt.scatter(X_train_std[y_km==1,0],X_train_std[y_km==1,1],c='orange',marker='o',edgecolor='orange',s=50,label='Cluster 2')
plt.scatter(X_train_std[y_km==2,0],X_train_std[y_km==2,1],c='lightblue',marker='v',edgecolor='lightblue',s=50,label='Cluster 3')
plt.scatter(km.cluster_centers_[:,0],km.cluster_centers_[:,1],c='red',marker='*',edgecolor='red',s=50,label='Centroids')
plt.legend(scatterpoints=1)

**Q. 3-2 (1.5 point) DBSCAN Clustering** 

In [ ]:
# Load the Iris dataset
iris = datasets.load_iris()
X_train = iris.data[:, [2, 3]]  # Using only the features (petal length and petal width)

# Feature Scaling
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=6, metric='euclidean')
y_dbscan = dbscan.fit_predict(X_train_std)

In [ ]:
# Visualize the clusters
plt.scatter(X_train_std[y_dbscan == 0, 0], X_train_std[y_dbscan == 0, 1], c='lightgreen', marker='s', edgecolor='black', s=50, label='Cluster 1')
plt.scatter(X_train_std[y_dbscan == 1, 0], X_train_std[y_dbscan == 1, 1], c='orange', marker='o', edgecolor='orange', s=50, label='Cluster 2')
plt.scatter(X_train_std[y_dbscan == 2, 0], X_train_std[y_dbscan == 2, 1], c='lightblue', marker='v', edgecolor='lightblue', s=50, label='Cluster 3')

plt.legend(scatterpoints=1)
plt.xlabel('Petal Length (standardized)')
plt.ylabel('Petal Width (standardized)')
plt.title('DBSCAN Clustering on Iris Dataset')
plt.show()

**Q.4 (1 point) PCA** 

In [ ]:
# Firt run the following code to get another data set.
from sklearn.datasets import make_moons
Xd, yd=make_moons(n_samples=200, noise=0.05, random_state=0)
plt.scatter(Xd[:,0],Xd[:,1])

db=DBSCAN(eps=0.2,min_samples=5,metric='euclidean')
y_db=db.fit_predict(Xd)
plt.scatter(Xd[y_db==0,0],Xd[y_db==0,1],c='lightgreen',marker='s',edgecolor='black',s=50,label='Cluster 1')
plt.scatter(Xd[y_db==1,0],Xd[y_db==1,1],c='orange',marker='o',edgecolor='orange',s=50,label='Cluster 2')
plt.legend()

In [ ]:
# Perform Kernel PCA
kpca = KernelPCA(n_components=1, kernel="rbf", gamma=12.5)
Xd_kpca = kpca.fit_transform(Xd)

# Visualize the results after Kernel PCA
plt.scatter(Xd_kpca[y_db == 0], np.zeros_like(Xd_kpca[y_db == 0]), c='lightgreen', marker='s', edgecolor='black', s=50, label='Cluster 1')
plt.scatter(Xd_kpca[y_db == 1], np.zeros_like(Xd_kpca[y_db == 1]), c='orange', marker='o', edgecolor='orange', s=50, label='Cluster 2')

plt.legend()
plt.xlabel('Principal Component 1 (Kernel PCA)')
plt.title('Kernel PCA Results after DBSCAN Clustering')
plt.show()

**Q.5 (2 points) Neural Networks** 

In [ ]:
# Run the following code for creating a data set of the XOR problem. 
np.random.seed(1)
x=np.random.uniform(low=-1, high=1, size=(200,2))
y=np.ones(len(x))

y[x[:,0]*x[:,1]<0]=0
# Split the training set into the training set and the validation set
x_train=x[:100,:]
y_train=y[:100]

x_valid=x[100:,:]
y_valid=y[100:]

In [ ]:
# Define the model
model = Sequential()

# Layer 1: Combined layer (input layer + hidden layer)
model.add(Dense(units=3, input_shape=(2,), activation='relu'))

# Layer 2: Hidden layer
model.add(Dense(units=5, activation='relu'))

# Layer 3: Hidden layer
model.add(Dense(units=4, activation='relu'))

# Layer 4: Output layer
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# Compile the model with specified hyperparameters
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

# Train the model
hist = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=200, batch_size=2, verbose=0)

# Visualization
history = hist.history
fig = plt.figure(figsize=(16, 4))

# Plot Training and Validation Loss
ax = fig.add_subplot(1, 3, 1)
plt.plot(history['loss'], lw=4)
plt.plot(history['val_loss'], lw=4)
plt.legend(['Train loss', 'Validation loss'], fontsize=15)
ax.set_xlabel('Epochs', size=15)

# Plot Training and Validation Accuracy
ax = fig.add_subplot(1, 3, 2)
plt.plot(history['binary_accuracy'], lw=4)
plt.plot(history['val_binary_accuracy'], lw=4)
plt.legend(['Train Acc', 'Validation Acc'], fontsize=15)
ax.set_xlabel('Epochs', size=15)

# Plot Decision Regions
ax = fig.add_subplot(1, 3, 3)
plot_decision_regions(X=x_valid, y=y_valid.astype(np.integer), clf=model)
ax.set_xlabel(r'$x_1$', size=15)
ax.xaxis.set_label_coords(1, -0.025)
ax.set_ylabel(r'$x_2$', size=15)
ax.yaxis.set_label_coords(-0.025, 1)

plt.show()